In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
import random
import datetime
import sys
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
def strQ2B(ustring):
    """全角转半角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:                              #全角空格直接转换            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248

        rstring += chr(inside_code)
    return rstring
    
def strB2Q(ustring):
    """半角转全角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 32:                                 #半角空格直接转化                  
            inside_code = 12288
        elif inside_code >= 32 and inside_code <= 126:        #半角字符（除空格）根据关系转化
            inside_code += 65248

        rstring += chr(inside_code)
    return rstring

In [11]:
etp_toggle = 'ETF'
# etp_toggle = 'LIP'    

In [14]:
# read in code list and create file
index_code_hk_stocks_pd = pd.read_excel('code_list_hk_all_' + etp_toggle + 's_v20181024.xlsx', header=None)
index_code_hk_stocks_ls = index_code_hk_stocks_pd[0].tolist()

columns_ls = ['index_code_hk', 'etf_name', 'listing_date', 'exp_ratio', 'suspend_status', 'suspend_date', 'replication_method', 'outstanding_shares', 'prev_close', 'lot_size']
results_filename = 'hkex-data-' + etp_toggle + '-v%s-supplement-2.csv' % datetime.date.today().strftime('%Y%m%d')

results_header_df = pd.DataFrame(columns = columns_ls)
results_header_df.to_csv(results_filename, header=True)       
print(results_filename + " created.")  

hkex-data-ETF-v20181129-supplement-2.csv created.


In [8]:
results_header_df

,index_code_hk,etf_name,listing_date,exp_ratio,suspend_status,suspend_date,replication_method,outstanding_shares,prev_close,lot_size


In [15]:
for i in [85, 91, 126, 143, 179, 180]:
# for i in range(len(index_code_hk_stocks_ls)):
    
    tic = datetime.datetime.now()
    
    try: 
        index_code_hk = index_code_hk_stocks_ls[i]
        index_page = 'https://sc.hkex.com.hk/TuniS/www.hkex.com.hk/market-data/securities-prices/exchange-traded-products/exchange-traded-products-quote?sym=' + str(index_code_hk) + '&sc_lang=zh-cn'
        driver = webdriver.PhantomJS() 
        driver.get(index_page)
        soup = BeautifulSoup(driver.page_source)

        etf_name = soup.find_all('p')[0].get_text()
        etf_name = etf_name if not None else ''
        etf_name = etf_name[:etf_name.rfind('(')].strip()
        etf_name = strQ2B(etf_name)
        etf_name = etf_name.replace('—', '-')
        
        listing_date = soup.find_all("td", "ano col_listingdate")[0].get_text()
        listing_year = listing_date[listing_date.find("年")-4:listing_date.find("年")]
        listing_month = listing_date[listing_date.find("年")+1:listing_date.find("月")]
        listing_day = listing_date[listing_date.find("月")+1:listing_date.find("日")]
        listing_date_output = str(listing_year + '/' + listing_month + '/' + listing_day)

        exp_ratio = soup.find_all('td', 'ano col_managefee')[0].get_text()
        exp_ratio = exp_ratio if not None else ''
        exp_ratio = exp_ratio[:exp_ratio.find('%')]
        
        suspend_tag = soup.find_all('span', 'susico')[0]
        suspend = 'suspend'
        
        if suspend in str(suspend_tag):
            suspend_status = 1
            
            suspend_date = soup.find_all("dt", "ico_data col_aum_date")[0].get_text()
            suspend_year = suspend_date[suspend_date.find("年")-4:suspend_date.find("年")]
            suspend_month = suspend_date[suspend_date.find("年")+1:suspend_date.find("月")]
            suspend_day = suspend_date[suspend_date.find("月")+1:suspend_date.find("日")]

            suspend_date_output = str(suspend_year + '/' + suspend_month + '/' + suspend_day)
        else:
            suspend_status = 0
            suspend_date_output = '-'

        replication_method = soup.find_all("td", "col_replicatemethod")[0].get_text()
        
        num_issued_shares = soup.find_all("td", "ano col_issued_shares")[0].get_text()
        num_issued_shares = num_issued_shares[:num_issued_shares.find("(")].strip().replace(",", "")
        
        prev_close = soup.find_all("dt", "ico_data col_prevcls")[0].get_text()
        
        lot_size = soup.find_all("dt", "ico_data col_lotsize")[0].get_text()
                
        df_tmp = pd.DataFrame([[index_code_hk, etf_name, listing_date_output, exp_ratio, suspend_status, suspend_date_output, replication_method, num_issued_shares, prev_close, lot_size]], columns = columns_ls)

        df_tmp.to_csv(results_filename, header=False, mode='a')
#         sys.stdout.flush()

    except:
        print(index_code_hk, ': error -------------------------')
        sys.stdout.flush()
    
    toc = datetime.datetime.now()
    print(i+1, '/', len(index_code_hk_stocks_ls), ' ', 'processed, time elapsed: ', toc - tic)
    sys.stdout.flush()

    rand = random.randint(2,10)
    print("About to sleep %d s" % rand)
    sys.stdout.flush()
    
    time.sleep(rand)

print("======= Finished =======")

86 / 185   processed, time elapsed:  0:00:50.200434
About to sleep 7 s
92 / 185   processed, time elapsed:  0:01:14.872062
About to sleep 2 s
127 / 185   processed, time elapsed:  0:01:12.314530
About to sleep 7 s
144 / 185   processed, time elapsed:  0:01:13.040966
About to sleep 8 s
180 / 185   processed, time elapsed:  0:03:29.151843
About to sleep 8 s
181 / 185   processed, time elapsed:  0:01:11.829812
About to sleep 3 s
======= Finished =======


In [ ]:
# test for new elements
index_page = 'https://sc.hkex.com.hk/TuniS/www.hkex.com.hk/market-data/securities-prices/exchange-traded-products/exchange-traded-products-quote?sym=2800&sc_lang=zh-cn'
driver = webdriver.PhantomJS()        
driver.get(index_page)
soup = BeautifulSoup(driver.page_source)

